In [1]:
import numpy as np
import pandas as pd 
import re
from sklearn.linear_model import LogisticRegression

In [2]:
# Read train and test data in to data dictionary.
data = {}
data["train"] = pd.read_csv("train.csv")
# data["test"] = pd.read_csv("test.csv")
# data["test"].head()

In [3]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
words = model.index2word

In [4]:
print(len(model.wv["computer"]))
print(len(model.vocab))
print(len(data["train"]))
data["train"].head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


300
3000000
404290


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
class FeaturePreparer:
    FEATURE_SIZE = 300
    def __init__(self):
        pass
    def IsNumber(self, input_string):
        if re.match("[0-9]+", input_string):
            return True
        else:
            return False
    def IsSymbol(self, input_string):
        return len(input_string) == 1 and input_string in [",./;'[]\\-=`~!@#$%^&*()_+{}|:\"<>?"]
    
    def GenerateWord2VecFeature(self, word):
        if word not in model.vocab:
            return np.zeros((1, 300)), True
        else:
            return model.wv[word], False
        
    def GenerateFeature(self, input_string):
        if type(input_string) != type(""):
            input_string = ""
            
        words, symbols, numbers = [], [], []
        for token in input_string.lower().split():
            if self.IsNumber(token):
                numbers.append(token)
            elif self.IsSymbol(token):
                symbols.append(token)
            else:
                words.append(token)
        feature_vec = np.array([[]])
        
        word_sum_vec = np.zeros((1, 300))
        total_words = 0
        for word in words:
            word_vec, is_zero = self.GenerateWord2VecFeature(word)
            word_sum_vec += word_vec
            if not is_zero:
                total_words += 1
        if total_words != 0:
            word_sum_vec /= 1.0 * total_words
        feature_vec = np.concatenate((feature_vec, word_sum_vec), axis = 1)
        
        return feature_vec
feature_preparer = FeaturePreparer()
# feature_preparer.GenerateFeature("computercdsaf")

In [11]:
class DataModelOnlyUseWord2Vec:
    def PrepareData(self):
        TRAIN_SPLIT = 85
        self.train_x = np.zeros((len(data["train"]) // 100 * TRAIN_SPLIT, FeaturePreparer.FEATURE_SIZE))
        self.train_y = []

        self.valid_x = np.zeros((len(data["train"]) - (len(data["train"]) // 100 * TRAIN_SPLIT), FeaturePreparer.FEATURE_SIZE))
        self.valid_y = []

        def GenerateDataPoint(question_1, question_2, is_duplicate):
            feature_1 = feature_preparer.GenerateFeature(question_1)
            feature_2 = feature_preparer.GenerateFeature(question_2)
            return feature_1 - feature_2
        for index, row in data["train"].iterrows():
            if index % 10000 == 0:
                print (index)
            if index < len(data["train"]) // 100 * TRAIN_SPLIT:
                self.train_x[len(self.train_y)] = GenerateDataPoint(row.question1, row.question2, row.is_duplicate)
                self.train_y.append(int(row.is_duplicate))
            else:
                self.valid_x[len(self.valid_y)] = GenerateDataPoint(row.question1, row.question2, row.is_duplicate)
                self.valid_y.append(int(row.is_duplicate))

        self.train_y = np.array(self.train_y)
        self.valid_y = np.array(self.valid_y)
#         print(self.train_x.shape, self.train_y.shape)
        print("Generate data model only use word2vec finished! Start to train!")
    def Train(self):
        clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(self.train_x, self.train_y)
        print(clf.score(self.train_x, self.train_y))
        prediction = clf.predict(self.valid_x)
        print (np.average(prediction == self.valid_y))

In [12]:
data_model_1 = DataModelOnlyUseWord2Vec()
data_model_1.PrepareData()
# data_model_1.Train()

0


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
Generate data model only use word2vec finished! Start to train!
